In [1]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation,Input, Merge, add
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import random

Using TensorFlow backend.


In [2]:
data = pd.read_csv('../Data/trainset_impute_onehot.csv')
x = data.drop(data.columns[[0, 1]], axis=1)
x = np.array(x)
y = data['target']
y = np.array(y)

In [5]:
1-np.mean(y)

0.96355248214081701

In [15]:
random.seed(867)
train_index = np.random.choice(range(data.shape[0]),int(0.8*data.shape[0]))
validate_index=set(range(data.shape[0]))-set(train_index)
validate_index=np.array(list(validate_index),dtype='int64')

In [16]:
x_train = x[train_index,:]
x_validate = x[validate_index,:]
y_train = y[train_index]
y_validate = y[validate_index]

In [17]:
case_index = [i for i,t in enumerate(y_train) if t == 1]
control_index = [i for i,t in enumerate(y_train) if t == 0]

In [18]:
new_control_index = np.random.choice(control_index,4*len(case_index))
new_index = np.concatenate((new_control_index,case_index))
new_index

array([128699, 286202, 451859, ..., 476111, 476142, 476152])

In [19]:
new_x_train = x_train[new_index,:]
new_y_train = y_train[new_index]

In [54]:
import keras.backend as K
import tensorflow as tf
def gini(actual, pred):
    l = K.shape(actual)
    val = np.arange(l)
    var = K.variable(value=val)
    temp = np.c_[actual, pred, ]
    b = K.cast_to_floatx(temp)
    c = b[np.lexsort((b[:,2], -1*b[:,1]))]
    totalLosses = K.sum(c[:,0])
    giniSum = K.sum(K.cumsum(c[:,0]))/totalLosses
    giniSum -= (l + 1) / 2.
    return giniSum / l

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

## new loss
## pred  actual  loss
##  0       1      9
##  1       0      1 
##  1/0     1/0    0
##  a=5,b=-4,c=0

def new_loss(actual,pred):
    return K.mean(5*K.square(pred-actual)-4*(pred-actual),axis=-1)

In [ ]:
def gini(actual, pred):
    nT = K.shape(actual)[-1]
    n = K.cast(nT, dtype='int32')
    inds = K.reverse(tf.nn.top_k(pred, n)[1], axes=[0])
    a_s = K.gather(actual, inds)
    a_c = K.cumsum(a_s)
    n = K.cast(nT, dtype=K.floatx())
    giniSum = K.cast(K.sum(a_c) / K.sum(a_s), dtype=K.floatx()) - (n + 1) / 2.0

    return giniSum / n


def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [55]:
model = Sequential()
model.add(Dense(300, activation='relu', input_dim=new_x_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=new_loss, optimizer=sgd, metrics=[gini_normalized])
model.fit(new_x_train, new_y_train, epochs=20, batch_size=128)

TypeError: a float is required

In [60]:
a=np.array([1,2,3])
b=np.array([4,5,6])
v=np.concatenate([a,b], axis=1)
np.reshape(v,(3,2))

AxisError: axis 1 is out of bounds for array of dimension 1